# F1 Podium Prediction

## Importing Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import LabelEncoder

## Initializing Dataset

In [56]:
dfStartingPos = pd.read_csv("assets/f1db-races-starting-grid-positions.csv")
dfRaceResults = pd.read_csv("assets/f1db-races-race-results.csv")
dfDriver = pd.read_csv("assets/f1db-drivers.csv")
dfFp1 = pd.read_csv("assets/f1db-races-free-practice-1-results.csv")
dfFp2 = pd.read_csv("assets/f1db-races-free-practice-2-results.csv")
dfFp3 = pd.read_csv("assets/f1db-races-free-practice-3-results.csv")
dfRaces = pd.read_csv("assets/f1db-races.csv")

C:\Users\Molina\AppData\Local\Temp\ipykernel_23440\2880382200.py:2: DtypeWarning: Columns (30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  dfRaceResults = pd.read_csv("assets/f1db-races-race-results.csv")


Dropping other columns for simplicity.  
Columns for dfStartingPos:
- positionDisplayOrder
- driverId  

Columns for dfRaceResults: 
- raceId 
- positionDisplayOrder  
- driverId  

Columns for dfFp1,dfFp2,dfFp3:  
- positionDisplayOrder
- driverId  

Columns for dfDriver:  
- id
- name
- totalPodiums
- totalPolePositions

Columns for dfRaces:
- id
- year

In [57]:
col1 = ['raceId','positionDisplayOrder','driverId']
dfStartingPos = dfStartingPos[col1]
dfRaceResults = dfRaceResults[col1]
dfFp1 = dfFp1[col1]
dfFp2 = dfFp2[col1]
dfFp3 = dfFp3[col1]
dfDriver = dfDriver[['id','name','totalPodiums','totalPolePositions']]
dfRaces = dfRaces[['id','year','grandPrixId']]

dfStartingPos = 25008 rows × 2 columns

dfRaceResults = 26771 rows x 2 columns

dfFp1 = 15316 rows × 2 columns

dfFp2 = 14979 rows × 2 columns

dfFp3 = 8290 rows × 2 columns

dfRaces = 1125 rows × 3 columns



In [61]:
dfStartingPos.rename(columns={"positionDisplayOrder":'startingPos'},inplace=True)
dfRaceResults.rename(columns={"positionDisplayOrder":'finishPos'},inplace=True)
dfFp1.rename(columns={"positionDisplayOrder":"fp1Pos"},inplace=True)
dfFp2.rename(columns={"positionDisplayOrder":"fp2Pos"},inplace=True)
dfFp3.rename(columns={"positionDisplayOrder":"fp3Pos"},inplace=True)
dfDriver.rename(columns={"id":'driverId'},inplace=True)
dfRaces.rename(columns={"id":"raceId"},inplace=True)

In [76]:
mergedDf1 = pd.merge(dfRaceResults,dfStartingPos,on=['raceId','driverId'],how='inner')
mergedDf2 = pd.merge(mergedDf1,dfFp1,on=['raceId','driverId'],how='inner')
mergedDf3 = pd.merge(mergedDf2,dfFp2,on=['raceId','driverId'],how='inner')
mergedDf4 = pd.merge(mergedDf3,dfFp3,on=['raceId','driverId'],how='inner')
mergedDf5 = pd.merge(mergedDf4,dfDriver, on=['driverId'],how='inner')
df = pd.merge(mergedDf5,dfRaces,on=['raceId'],how='inner')

In [77]:
df

,raceId,finishPos,driverId,startingPos,fp1Pos,fp2Pos,fp3Pos,name,totalPodiums,totalPolePositions,year,grandPrixId
0,706,1,ralf-schumacher,3,8,7,1,Ralf Schumacher,27,6,2003,europe
1,706,2,juan-pablo-montoya,4,11,4,2,Juan Pablo Montoya,30,13,2003,europe
2,706,3,rubens-barrichello,5,12,2,10,Rubens Barrichello,68,14,2003,europe
3,706,4,fernando-alonso,8,5,12,8,Fernando Alonso,106,22,2003,europe
4,706,5,michael-schumacher,2,10,6,9,Michael Schumacher,155,68,2003,europe
...,...,...,...,...,...,...,...,...,...,...,...,...
7692,1123,16,liam-lawson,14,20,15,18,Liam Lawson,0,0,2024,las-vegas
7693,1123,17,esteban-ocon,11,14,12,14,Esteban Ocon,4,0,2024,las-vegas
7694,1123,18,valtteri-bottas,19,16,11,17,Valtteri Bottas,67,20,2024,las-vegas
7695,1123,19,alexander-albon,17,13,20,7,Alexander Albon,2,0,2024,las-vegas


In [78]:
df.isnull().sum()

raceId                0
finishPos             0
driverId              0
startingPos           0
fp1Pos                0
fp2Pos                0
fp3Pos                0
name                  0
totalPodiums          0
totalPolePositions    0
year                  0
grandPrixId           0
dtype: int64